Librairies

In [863]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import ttest_ind
from scipy.stats import f_oneway
import statsmodels.api as sm
import plotly.graph_objects as go
import statsmodels.formula.api as smf
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from joblib import dump
from joblib import load

1. Contexte

Étant cycliste passionné, j'ai souhaité modéliser un indice de performance à l'aide de la vitesse moyenne et à partir de différents facteurs comme le dénivelé, la longueur, le fait de rouler en groupe ou non, etc... 
En effet, n'ayant pas de capteur de puissance, j'ai voulu me créer un indicateur de performance avec les données disponibles pour suivre mon évolution et ma forme tout au long de l'année.
Certaines données, comme par exemple les données météorologiques ne sont pas disponibles. Cela peut créer un biais car les performances sportives sont impactées par les conditions climatiques (vent, froid, fortes chaleurs, humidité).
Pour réaliser ce modèle, j'ai utilisé des données personelles récupérées à partir d'archives téléchargeables sur https://www.strava.com et des données sur les segments strava que j'ai retranscrites sur un fichier .csv.

2. Descriptif des données

A. Dataset Strava

Ce dataset contient les données générales sur les activités sportives que l'on poste sur le réseau social sportif Strava. On peut le télécharger depuis le site officiel de l'application. Ce dataset contient 510 observations mais ce nombre est susceptible d'évoluer lors de l'ajout de nouvelles données. La première observation date du 4 mai 2019 et la plus récente du 25 janvier 2026.

Variables principales :

    Activity Date => date de l'activité  
    Activity Name => nom de l'activité  
    Activity Type => type de sport de l'activité  
    Elapsed Time => durée de l'activité (en secondes)  
    Distance => distance de l'activité (en km)  
    Max Heart Rate => fréquence cardiaque maximum au cours de l'activité  
    Moving Time => durée de l'activité, en excluant les pauses (en secondes)  
    Elevation Gain => dénivelé positif (en m)  

Ces données ne prennent ni en compte la météo, ni la qualité du revêtement de la route, ni la difficulté des différentes pentes sur le parcours (si on a 500m de dénivelé, on ne sait pas si on a eu 10 côtes de 1km à 5%, une côte de 5km à 10%, ou une côte de 10km à 5% par exemple, alors que l'impact sur la vitesse peut être différent). La détection du fait de rouler en groupe ou seul se fait à l'aide du nom de l'activité (contient ou non "avec" ou "ACE"), mais certaines sorties en groupe ne contiennent pas ces éléments dans le nom de l'activité. On ne tient pas compte du dénivelé négatif car les parcours se font pour la quasi-totalité des activités d'un point A à ce même point A. Pour les parours où ce n'est pas le cas, l'altitude d'arrivée est toujours la même (+/- 50m) à l'altitude de départ. 

B. Dataset segments

Ce dataset contient les données sur des segments Strava sélectionnés pesonellement. Les données sont récupérées sur le détail des activités postées sur Strava et retranscrites dans un fichier csv. Ce dataset contient les données de segments présentant à minima 100m de dénivelé positif, ce qui correspond, à ce jour, à 208 observations.

Variables principales :

    Col => nom du segment  
    Dénivelé => dénivelé positif total du segment (en m)  
    Longueur => longueur total du segment (en km)  
    Pente => pourcentage moyen de la pente sur le segment  
    Durée => temps pour parcourir le segment (en secondes)  
    Vitesse => vitesse moyenne sur le segment (en km/h)  

Ces donées ne prennent en compte ni la météo, ni les efforts fournis avant le segment, ni la qualité de revêtement de la route, ce qui peut provoquer du biais lors de la modélisation.

3. Import et restructuration des données

In [864]:
# Import des deux dataset

Strava = pd.read_csv('D:/Documents/R/Vélo/Data_Strava/activities.csv')
segments = pd.read_csv("D:/Documents/R/Vélo/Cyclisme/Liste_cols.csv", encoding="UTF8", quotechar=",")

In [865]:
# Recalcul de la vitesse moyenne à partir du temps de déplacement (en km/h)

Strava['Vitesse_moyenne'] = Strava.Distance/(Strava['Moving Time']/3600)

In [866]:
# Sélection des variables pertinentes

Strava = Strava[['Activity Date', 'Activity Name', 'Activity Type','Elapsed Time','Distance.1','Elevation Gain','Vitesse_moyenne','Moving Time']]
Strava = Strava.rename(columns = {'Distance.1':'Distance_m'})

In [867]:
# Sélection des activités de cyclisme sur route

Strava = (
    Strava
    .loc[Strava['Activity Type'] == 'Ride']
    .copy()
)

In [868]:
# Identification des sorties en groupe

patterns = {
    'pere': r'père|pere',
    'nausicaa': r'nausicaa|naunau|copine',
    'avec': r'père|pere|loïc|loic|nausicaa|mère|yann|naunau|hugo|balade|promenade|arnaud|mere|avec|ACE'
}

for col, regex in patterns.items():
    Strava[col] = (
        Strava['Activity Name']
        .str.contains(regex, case=False, na=False)
        .astype('category')
    )

In [869]:
# Transformation et ajout de variables

Strava = Strava.assign(
    **{'annee' : Strava["Activity Date"].str.split(',', expand = True)[1].str[1],
       'elevation_km' : (Strava["Elevation Gain"]/Strava["Distance_m"])*1000, # cette variable apporte l'information sur la difficulté du parcours
       'date' : pd.to_datetime(Strava["Activity Date"], format = '%b %d, %Y, %I:%M:%S %p')} 
)

4. Analyse exploratoire des données

In [870]:
histo_vitesse = px.histogram(Strava, x = 'Vitesse_moyenne',title="Distribution de la vitesse moyenne",
                            labels={
                                "Vitesse_moyenne" : "Vitesse moyenne (km/h)"
                                })
histo_vitesse

In [871]:
# Relation entre la vitesse moyenne, le fait de rouler en groupe et le dénivelé

vitesse_denivele = px.scatter(Strava, x = 'Vitesse_moyenne',y = 'Elevation Gain', color = Strava["avec"], hover_data=['Activity Name','Distance_m','Elevation Gain'])
vitesse_denivele

C:\Users\tronc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\plotly\express\_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [872]:
vitesse_denivele_km = px.scatter(Strava, x = 'Vitesse_moyenne',y = 'elevation_km', color = Strava["avec"], hover_data=['Activity Name','Distance_m','Elevation Gain'])
vitesse_denivele_km

C:\Users\tronc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\plotly\express\_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Hypothèses après l'observation du graphique : vitesse moyenne plus faible quand il y a plus de dénivelé et quand je roule en groupe. Le but va être de vérifier rapidement ces hypothèses pour aider à sélectionner les variables qui seront utilisées dans la modélisation

In [873]:
# Tableau de corrélation et test statistique

cols = [
    'Vitesse_moyenne',
    'Distance_m',
    'Elevation Gain',
    'avec',
    'elevation_km'
]

# Pearson

corr = Strava[cols].corr()
corr

,Vitesse_moyenne,Distance_m,Elevation Gain,avec,elevation_km
Vitesse_moyenne,1.000000,0.139089,-0.090566,-0.484683,-0.327522
Distance_m,0.139089,1.000000,0.825355,0.149257,0.293924
Elevation Gain,-0.090566,0.825355,1.000000,0.081329,0.711018
avec,-0.484683,0.149257,0.081329,1.000000,0.063335
elevation_km,-0.327522,0.293924,0.711018,0.063335,1.000000


La variable "Elevation Gain" est faiblement corrélée avec la variable "Vitesse_moyenne" mais fortement corrélée avec la variable "Distance_m". Il n'est pas pertinent de la choisir comme variable à utiliser dans le modèle. À la place on va préférer utiliser la variable "elevation_km" qui est un meilleur indicateur de la difficulté du parcours (représente le dénivelé positif (en mètres) par kilomètre parcouru)

On va tester l'hypothèse que la vitesse moyenne est identique selon si je roule seul ou en groupe (moyenne solo = moyenne groupe).

In [874]:
solo = Strava[Strava['avec'] == False]['Vitesse_moyenne'] # écrire hypothèses
groupe = Strava[Strava['avec'] == True]['Vitesse_moyenne']

ttest_ind(solo, groupe, equal_var=False)

TtestResult(statistic=11.082702130998442, pvalue=2.1010918359350333e-24, df=322.71011501374795)

La pvalue est < 0.001. On rejette l'hypothèse qu'il n'y a pas de différence lorsque je roule seul ou en groupe. Le fait de rouler en groupe fait baisser la vitesse moyenne. Le résultat peut paraître contre-intuitif car on roule plus vite en groupe que seul, mais la plupart de temps que je roule en groupe, c'est avec des personnes ayant un niveau plutôt débutant. Ce résultat du test me paraît donc logique.

5. Nettoyage des données

In [875]:
np.random.seed(42)

Vérification des NA et des valeures aberrantes

In [876]:
Strava.isna().mean().sort_values(ascending=False)

Elevation Gain     0.009091
elevation_km       0.009091
Activity Date      0.000000
Activity Name      0.000000
Activity Type      0.000000
Elapsed Time       0.000000
Distance_m         0.000000
Vitesse_moyenne    0.000000
Moving Time        0.000000
pere               0.000000
nausicaa           0.000000
avec               0.000000
annee              0.000000
date               0.000000
dtype: float64

2 variables présentent des valeurs manquantes (1 réelle étant donné que la variable "elevation_km" est calculée à partir de "Elevation Gain")

In [877]:
Strava.describe()

,Elapsed Time,Distance_m,Elevation Gain,Vitesse_moyenne,Moving Time,elevation_km,date
count,440.000000,440.000000,436.000000,440.000000,440.000000,436.000000,440
mean,8347.793182,48890.551591,682.508028,23.255083,7658.672727,13.130454,2023-04-25 16:52:28.149999872
min,901.000000,5041.000000,12.400000,10.405710,851.000000,0.570563,2019-05-04 08:44:42
25%,4995.000000,30251.450000,364.950000,21.146471,4675.500000,10.275565,2021-10-11 21:25:15.500000
50%,8085.500000,46197.300000,584.300000,23.844892,7499.500000,12.926414,2023-09-06 06:56:10
75%,11092.500000,65297.400000,973.675000,26.131968,10315.500000,15.886210,2024-09-28 20:31:41.249999872
max,36149.000000,132760.600000,3022.900000,32.881556,21734.000000,35.796276,2025-11-22 07:49:02
std,4325.578938,23607.145575,473.674426,3.609214,3651.721729,5.488180,NaN


On ne constate pas d'aberrations dans les données. Les activités étant consultables sur l'application, je n'avais pas non plus remarqué d'aberrations dessus. Les erreurs peuvent cependant apparaître lors du téléchargement de l'archive contenant les données.

In [878]:
Strava.loc[Strava.elevation_km.isna()] # Identification des lignes avec des valeurs #NA

,Activity Date,Activity Name,Activity Type,Elapsed Time,Distance_m,Elevation Gain,Vitesse_moyenne,Moving Time,pere,nausicaa,avec,annee,elevation_km,date
31,"Dec 18, 2019, 3:44:51 PM",Sortie à vélo dans l'après-midi,Ride,3529,21757.4,NaN,22.781495,3437.0,False,False,False,2,NaN,2019-12-18 15:44:51
32,"Dec 19, 2019, 7:20:20 AM",Sacré vent de face !,Ride,3930,21416.8,NaN,19.768146,3899.0,False,False,True,2,NaN,2019-12-19 07:20:20
33,"Dec 25, 2019, 3:47:53 PM",Sortie à vélo dans l'après-midi,Ride,3241,21659.4,NaN,24.111317,3234.0,False,False,False,2,NaN,2019-12-25 15:47:53
34,"Dec 28, 2019, 1:44:02 PM",25km avec un pote 😀,Ride,9752,45663.9,NaN,19.501246,8429.0,False,False,True,2,NaN,2019-12-28 13:44:02


In [879]:
Strava_modele = Strava.loc[~Strava['elevation_km'].isna()].copy() # On enlève les lignes où il manque le dénivelé.

In [880]:
Strava_modele['avec'] = Strava_modele['avec'].astype(int) # récupération de la variable "avec" au format numérique pour la modélisation

J'aurais pu compléter les données manquantes de plusieurs façons. La première aurait été de récupérer la distance moyenne et le dénivelé moyen, puis d'adapter proportionnellement le dénivelé à la distance. J'aurais également pu estimer le dénivelé à l'aide d'un petit modèle de régression simple.
Étant donné que seules 4 sorties sont concernées (sur 400), et qu'elles sont anciennes (j'ai débuté le vélo en 2019, et la progression étant rapide au début, ces données sont moins pertinentes), j'ai donc préféré ne pas les garder.

Certaines activités présentent des performances inhabituelles car elles ont été réalisées avec d'autres personnes avec un niveau débutant. Celles-ci sont indentifiées à l'aide de la variable "avec" lorsqu'elle est égale à 1.
Lors de la phase de test de la modélisation, cela permettra d'avoir un premier indicateur sur un potentiellement sur-entraînement du modèle : si le modèle prédit correctement la vitesse moyenne sur une sortie avec une personne très débutante, ou par exemple en ayant eu vent de dos tout le long lors d'un aller simple sur du plat, on pourrait alors soupçonner un overfitting du modèle.

Nous avons déjà ajouté le dénivelé par kilomètre comme variable d'intérêt pour notre modèle. Cependant, un autre élément est important dans les performances sportives : la période de l'année. Les performances sont généralement meilleures à partir de la fin du printemps, jusqu'au milieu de l'automne. Nous allons donc ajouter le mois de l'année au dataset.

In [881]:
Strava_modele['mois'] = Strava_modele['date'].dt.month.astype("category")
mois_fr = {
    1: 'janvier',
    2: 'février',
    3: 'mars',
    4: 'avril',
    5: 'mai',
    6: 'juin',
    7: 'juillet',
    8: 'août',
    9: 'septembre',
    10: 'octobre',
    11: 'novembre',
    12: 'décembre'
}
Strava_modele['mois_nom'] = Strava_modele['mois'].map(mois_fr)

6. Modélisation

Nous cherchons à modéliser la vitesse moyenne sur une sortie selon les différentes variables retenues afin de pouvoir prédire la durée estimée pour effectuer le parcours, comparer la vitesse estimée et la vitesse réelle pour évaluer un état de forme, ou encore pour pouvoir cibler une allure inférieure à la vitesse prédite si l'on souhaite réaliser un entraînement d'endurance ou de la récupération.

La variable d'intérêt est donc la vitesse moyenne. Afin de l'expliquer, nous allons réaliser dans un premier temps une régression linéaire multiple contenant les variables suivantes :

    - Distance_m  
    - avec  
    - elevation_km  
    - mois_nom (sous forme de dummies)  

Ce modèle est simple et permet facilement d'interpréter les résultats et ne demande pas une standardisation des données.

    1er modèle avec StatsModele

Dans un 1er temps, je fais faire une régression multiple simple issue de la librairie statsmodels afin d'évaluer la pertinence des variables explicatives (à l'aide des pvalues). Ensuite, une fois que les variables à utiliser seront validées, je referai un modèle à l'aide de scikit learn.

In [882]:
split = int(len(Strava_modele) * 0.75) # proportion du dataset pour l'entrainement (peut être faire un tirage aléatoire)

Strava_modele['Elev_c'] = Strava_modele['elevation_km'] - Strava_modele['elevation_km'].mean() # on centre la variable pour réduire la colinéarité entre la forme linéaire de la variable et la forme quadratique
Strava_modele['Elev_c2'] = Strava_modele['Elev_c'] ** 2

train = Strava_modele.iloc[:split]
test  = Strava_modele.iloc[split:]

In [883]:
formula = 'Vitesse_moyenne ~ Distance_m + avec + Elev_c + Elev_c2' # après des tests, le mois de l'année n'était finalement pas significatif (même en interraction avec "avec" ou le dénivelé par km), je l'ai donc enlevé.

model = smf.ols(formula, data=train).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        Vitesse_moyenne   R-squared:                       0.427
Model:                            OLS   Adj. R-squared:                  0.420
Method:                 Least Squares   F-statistic:                     59.91
Date:                Wed, 18 Feb 2026   Prob (F-statistic):           8.86e-38
Time:                        15:40:30   Log-Likelihood:                -794.74
No. Observations:                 327   AIC:                             1599.
Df Residuals:                     322   BIC:                             1618.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     22.0919      0.437     50.563      0.0

Plusieurs tests de variables ont été effectués : avec le centrage de la variable "elev_km" et l'intégration de sa forme quadratique, la performance du modèle a augmenté et la significativité de ces deux coefficients par rapport à la forme non centrée a également augmentée. Les mois se sont révélés non significatifs d'un point de vue statistique.

In [884]:
y_test = test['Vitesse_moyenne']

y_pred = model.predict(test)

In [885]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2   = r2_score(y_test, y_pred)

print("RMSE test :", rmse)
print("R² test   :", r2)

RMSE test : 2.7226569104355915
R² test   : 0.40316589084250354


Le R² du set de test est proche de celui du set de entraînement, il n'y a pas d'overfitting. Dans un premier temps, la proportion des données d'entraînement était de 0.8, et l'écart entre le R² du set de test et du set d'entraînement était plus élevé (0.37 face à 0.44). J'ai donc ajusté la proportion d'entraînement à 0.75.

    2ème modèle avec Scikit Learn

In [886]:
y = Strava_modele.Vitesse_moyenne
X = Strava_modele[['Distance_m','avec','Elev_c','Elev_c2']] 

Séparation du dataset pour l'entrainement et le test du modèle.

In [887]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.25,
    random_state = 70
)

In [888]:
modele_prediction_vitesse = sklearn.linear_model.LinearRegression()
modele_prediction_vitesse.fit(X_train,y_train)
y_pred = modele_prediction_vitesse.predict(X_test)

In [889]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

2.5266405868545525

In [890]:
coef = pd.Series(
    modele_prediction_vitesse.coef_,
    index=X.columns
).sort_values()
coef

avec         -3.771663
Elev_c       -0.258244
Elev_c2      -0.009698
Distance_m    0.000051
dtype: float64

In [891]:
resultat = X_test
resultat['true_speed'] = y_test
resultat['predicted_speed'] = y_pred
resultat['erreur'] = resultat.predicted_speed - resultat.true_speed

In [892]:
resultat['ID'] = resultat.index
Strava_modele['ID'] = Strava_modele.index
resultat = pd.merge(Strava_modele[['ID','Activity Name','Elevation Gain']],resultat,on='ID')
resultat[['Activity Name','Elevation Gain','Distance_m','true_speed','predicted_speed','erreur']].sort_values(by = 'erreur')

,Activity Name,Elevation Gain,Distance_m,true_speed,predicted_speed,erreur
79,"Repas chez les grands parents, vent de dos tou...",713.8,62827.4,32.051020,26.287081,-5.763940
83,"Duathlon Vautourman, relais Vélo",907.0,27684.1,20.708229,15.248143,-5.460086
82,Afternoon Ride,265.0,16135.8,27.757170,22.507066,-5.250104
97,Evening Ride,226.0,17117.8,28.216216,23.494258,-4.721958
69,Massif de la Malepère,768.6,74415.3,27.519622,23.331487,-4.188135
...,...,...,...,...,...,...
43,Tour découverte avec Hugo,572.1,42703.2,16.899736,20.985775,4.086039
105,🇫🇷🇮🇹🇸🇮 Tour du Lac d’Iseo + montée de Brione,1264.0,110048.6,24.368311,28.684282,4.315971
75,Balade avec Nausicaa,12.4,20352.8,15.802416,21.620566,5.818151
60,Balade avec Nausicaa,388.1,31007.6,14.329738,20.609601,6.279863


In [893]:
px.histogram(resultat,'erreur')

In [894]:
graph_erreur = px.scatter(resultat,x='true_speed',y='predicted_speed',color = 'erreur', hover_data=['Activity Name','Distance_m','Elevation Gain'])
graph_erreur.add_trace(
    go.Scatter(
        x=[11, 33],
        y=[11, 33],
        mode='lines',
        line=dict(color='red', dash='dash'),
        name='y = x'
    )
)


graph_erreur.show()

In [895]:
resultat['erreur_abs'] = resultat['erreur'].abs()
resultat.groupby('avec')['erreur_abs'].mean()

avec
0    1.966479
1    2.098439
Name: erreur_abs, dtype: float64

In [896]:
resultat.groupby('avec')['erreur_abs'].var()

avec
0    1.784543
1    3.131245
Name: erreur_abs, dtype: float64

On constate que la variabilité des erreurs est beaucoup plus élevée lorque "avec" = 1, c'est à dire lorsque je roule en groupe. Ce résultat n'est pas surprenant car cela regroupe des sorties avec des personnes variées (débutants, groupe cycliste de ma villes, amis plus expérimentés). Il aurait fallut créer plus de catégories afin d'améliorer le modèle, mais le nombre de données sur certaines catégories aurait été faible, même en faisant un groupement sur toutes les sorties avec un ami expérimenté, par exemple.

    2ème modèle

Nous pouvons utiliser un modèle plus complexe afin de comparer les résultats et vérifier si la performance augmente. Nous cherchons à prédire la vitesse moyenne, qui est une variable numérique continue. De plus, nos variables ont déjà été sélectionnées et nous avons du en transformer afin de réduire la colinéarité (la variable "elevation_km" et sa forme quadratique). L'utilisation de la regression Ridge garde toutes les variables que nous avons selectionné au préalable, et est efficace contre les problèmes de multicolinéarité.

Le modèle Ridge réduit la taille des coefficients. Cette pénalisation dépend de l’échelle des variables. Il est donc nécessaire de standardiser les données avant l’apprentissage.

Après avoir réalisé le modèle précédent pas à pas, nous utiliserons une pipeline pour automatiser les étapes. Les données sont d’abord standardisées, puis, plusieurs valeurs de alpha, comprises entre 0,0001 et 1000, sont testées. Pour chaque valeur, une validation croisée à 5 groupes est effectuée afin de sélectionner le meilleur modèle.

In [897]:
y = Strava_modele.Vitesse_moyenne
X = Strava_modele.assign(
    elevation_km_2 = Strava_modele['elevation_km'] ** 2
)[['Distance_m','avec','elevation_km','elevation_km_2']]


Nous utiliserons les variables initiales, car la fonction StandardScaler() centre et réduit automatiquement les données, rendant inutile un centrage manuel.

In [898]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.25,
    random_state = 70
)

In [899]:
model_ridge_CV = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", RidgeCV(alphas=np.logspace(-3, 3, 50), cv=5))
])

model_ridge_CV.fit(X_train, y_train)

best_alpha = model_ridge_CV.named_steps["ridge"].alpha_
print("Meilleur alpha :", best_alpha)

Meilleur alpha : 10.985411419875572


Maintenant que nous avons trouvé le meilleur alpha, nous pouvons construire le modèle final à partir de cette valeur d'alpha.

In [900]:
model_final = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=best_alpha))
])

model_final.fit(X_train, y_train)
y_pred = model_final.predict(X_test)

In [901]:
rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred))
r2_ridge  = r2_score(y_test, y_pred)

print("RMSE test :", rmse_ridge)
print("R² test   :", r2_ridge)

RMSE test : 2.515339468807346
R² test   : 0.42617471643951255


In [902]:
resultat_ridge = X_test
resultat_ridge['true_speed'] = y_test
resultat_ridge['predicted_speed'] = y_pred
resultat_ridge['erreur'] = resultat_ridge.predicted_speed - resultat_ridge.true_speed

In [903]:
resultat_ridge['ID'] = resultat_ridge.index
Strava_modele['ID'] = Strava_modele.index
resultat_ridge = pd.merge(Strava_modele[['ID','Activity Name','Elevation Gain']],resultat_ridge,on='ID')
resultat_ridge[['Activity Name','Elevation Gain','Distance_m','true_speed','predicted_speed','erreur']].sort_values(by = 'erreur')

,Activity Name,Elevation Gain,Distance_m,true_speed,predicted_speed,erreur
79,"Repas chez les grands parents, vent de dos tou...",713.8,62827.4,32.051020,26.142446,-5.908574
82,Afternoon Ride,265.0,16135.8,27.757170,22.454427,-5.302744
83,"Duathlon Vautourman, relais Vélo",907.0,27684.1,20.708229,15.901422,-4.806807
97,Evening Ride,226.0,17117.8,28.216216,23.420022,-4.796195
69,Massif de la Malepère,768.6,74415.3,27.519622,23.332629,-4.186993
...,...,...,...,...,...,...
105,🇫🇷🇮🇹🇸🇮 Tour du Lac d’Iseo + montée de Brione,1264.0,110048.6,24.368311,28.459012,4.090701
43,Tour découverte avec Hugo,572.1,42703.2,16.899736,21.024112,4.124376
75,Balade avec Nausicaa,12.4,20352.8,15.802416,22.007632,6.205216
60,Balade avec Nausicaa,388.1,31007.6,14.329738,20.668626,6.338888


In [904]:
resultat_ridge['erreur_abs'] = resultat_ridge['erreur'].abs()
resultat_ridge.groupby('avec')['erreur_abs'].mean()

avec
0    1.938217
1    2.096489
Name: erreur_abs, dtype: float64

In [905]:
resultat_ridge.groupby('avec')['erreur_abs'].var()

avec
0    1.767921
1    3.182768
Name: erreur_abs, dtype: float64

On remarque que les performances sont légèrement meilleures avec le modèle Ridge (R² de 0.43 au lieu de 0.40 sur le dataset test), avec également une variabilité légèrement réduite des erreurs lorsque je roule seul. 
Cependant, la différence de performance des deux modèles reste relativement proche (écart de 2 points pour le R²).  
Cela peut venir du fait que lors de l'entraînement du modèle linéaire multivarié, nous avons déjà corrigé un problème de colinéarité. Étant donné que le modèle Ridge, arrive à gérer la colinéarité des variables explicatives en réduisant les coefficients sur certaines variables, si nous n'avions pas géré ce problème dans le premier modèle, l'amélioration des performances sur le modèle Ridge aurait pu être plus marquée.  

Selon les besoins, s’arrêter au modèle linéaire multivarié peut être préférable : le gain en temps et en ressources peut compenser le gain marginal de performance apporté par la régularisation Ridge.

7. Calcul d'un indicateur de forme

Étant donné que le but du projet est de pouvoir estimer ma forme ou ma progression à partir de la vitesse moyenne, ce sera l'estimation de cette dernière qui sera utilisée pour l'indicateur de forme. Pour cela, nous allons récupérer les coefficients du dernier modèle.

In [906]:
dump(model_final, "performance_cyclisme.pkl") # le but final étant de programmer une petite application, il est nécessaire de sauvegarder le modèle.

['performance_cyclisme.pkl']

In [907]:
predicteur = load("performance_cyclisme.pkl")

In [908]:
Distance_km = float(input("Distance (km) : "))
Dénivelé = float(input("Dénivelé (m) : "))
avec = int(input("Avec une autre personne(0 si non, 1 si oui) : "))
elevation_km = Dénivelé/Distance_km
elevation_km_2 = elevation_km**2
Distance_m = Distance_km*1000

elevation_km_2 = elevation_km ** 2

X_predict = pd.DataFrame([{
    "Distance_m": Distance_m,
    "avec": avec,
    "elevation_km": elevation_km,
    "elevation_km_2": elevation_km_2
}])


In [909]:
prediction = predicteur.predict(X_predict)

print("Vitesse prédite :", round(prediction[0], 2), "km/h")

Vitesse prédite : 23.46 km/h


In [913]:
Vitesse_reelle = int(input("Vitesse réalisée : "))
Vitesse_predite = round(prediction[0], 2)

In [918]:
def indicateur_de_performance(vitesse_reelle, vitesse_predite):
    ecart = vitesse_reelle - vitesse_predite

    if ecart >= 2 :
        forme = "Très bonne" 
    elif (ecart < 2 and ecart >=1) :
        forme = "Bonne"
    elif ecart < -2 :
        forme = "Très mauvaise"
    elif (ecart <-1 and ecart >= -2) :
        forme = "Mauvaise"
    else : forme = "Normale"

    return("La forme actuelle est : " + str(forme))

In [919]:
indicateur_de_performance(Vitesse_reelle,Vitesse_predite)

'La forme actuelle est : Normale'